In [ ]:
import datetime

from pydantic import ConfigDict
import polars as pl
from tqdm import tqdm
import matplotlib.pyplot as plt

import stock
from stock.algorithm.market import is_limit_high

In [ ]:
# 細かく利益確定していくsimulation

In [ ]:
stacked_df = stock.watchlist.v1.get_watch_list_all()

In [ ]:
class StopCondition(stock.simulation.base_condition.BaseCondition):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    # parameter
    max_loss_rate: float = 0.08
    sell_rate: float = 0.02
    max_days: int = 7 * 2
    # results
    buying_price: float = -1
    buying_date: datetime.date = datetime.date.today()
    selling_price: float = -1
    selling_date: datetime.date = datetime.date.today()
    # internal
    loss_cut_price: float = -1
    target_price: float = -1
    index: int = -1
    df: pl.DataFrame = pl.DataFrame()

    def reset_results(self):
        self.buying_price = -1
        self.buying_date = datetime.date.today()
        self.selling_price = -1
        self.selling_date = datetime.date.today()
        self.loss_cut_price = -1
        self.target_price = -1
        self.index = -1
        self.df = pl.DataFrame()

    def set_start(self, src_df: pl.DataFrame, start_date: datetime.date) -> float:
        #print(src_df)
        self.reset_results()
        df = src_df.filter(pl.col("date") >= start_date).sort(pl.col("date"))
        if len(df) < 15:
            return -1
        
        if df["date"][0] - start_date > datetime.timedelta(days=10):
            return -1
        
        if is_limit_high(df["close"][0], df["open"][1]):
            return -1
        
        self.buying_price = df["open"][1]
        self.buying_date = df["date"][1]

        self.loss_cut_price = self.buying_price * (1 - self.max_loss_rate)
        self.target_price = self.buying_price * (1 + self.sell_rate)
        self.df = df
        self.index = 1
        return self.buying_price

    def run_simulation(self) -> float:

        if self.df["date"][self.index] - self.buying_date > datetime.timedelta(days=self.max_days):
            self.selling_date = self.df["date"][self.index]
            self.selling_price = self.df["open"][self.index]
        elif self.df["low"][self.index] < self.loss_cut_price:
            self.selling_date = self.df["date"][self.index]
            self.selling_price = min(self.df["open"][self.index], self.loss_cut_price)
        elif self.df["high"][self.index] > self.target_price:
            self.selling_date = self.df["date"][self.index]
            self.selling_price = max(self.df["open"][self.index], self.target_price)
        # else:
        #     self.selling_date = self.df["date"][self.index]
        #     self.selling_price = self.df["close"][self.index]

        self.index += 1
        return self.selling_price

In [ ]:
class CustomStopCondition(stock.simulation.simulate.BaseCondition):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    # 入力パラメータ
    max_loss_rate: float = 0.08  # 買値からの最大損失率
    trailling_stop_rate: float = 0.1  # ここまで値下がりしたら売る
    sell_rate: float = 0.2  # ここまで値上がりしたら半分売る
    max_days: int = 7 * 2  # 最大保持日数
    total_max_days: int = 7 * 4  # 最大保持日数
    # 結果変数
    buying_price: float = -1
    buying_date: datetime.date = datetime.date.today()
    selling_price: float = -1
    selling_date: datetime.date = datetime.date.today()
    # 内部計算用変数
    loss_cut_price: float = -1
    profit_fixed_price: float = -1
    reach_target_price: bool = False
    target_selling_price: float = -1
    highest_updated: bool = False
    index: int = -1
    df: pl.DataFrame = pl.DataFrame()
    src_df: pl.DataFrame = pl.DataFrame()

    def reset_results(self):
        self.buying_price = -1
        self.buying_date = datetime.date.today()
        self.selling_price = -1
        self.selling_date = datetime.date.today()
        self.loss_cut_price = -1
        self.profit_fixed_price = -1
        self.reach_target_price = False
        self.target_selling_price = -1
        self.highest_updated = False
        self.index = -1
        self.df = pl.DataFrame()
        self.src_df = pl.DataFrame()

    def set_start(self, src_df: pl.DataFrame, start_date: datetime.date) -> float:
        self.reset_results()
        df = src_df.filter(pl.col("date") >= start_date).sort(pl.col("date"))
        prev_df = src_df.filter(pl.col("date") < start_date).sort(pl.col("date"))
        if len(df) <= 30:
            return -1

        if df["date"][0] - start_date > datetime.timedelta(days=10):
            return -1

        # stop高は回避
        if is_limit_high(df["close"][0], df["open"][1]):
            return -1

        # ベースから上離れしすぎている場合はスキップ
        if prev_df["close"][-1] * 1.4 < df["open"][1]:
            return -1

        # 前日終値から下がりすぎている場合は買わない
        if df["open"][1] < prev_df["close"][-1] or df["open"][1] < df["close"][0] * 0.9:
            return -1

        #self.buying_price = df["open"][1]
        self.buying_price = df["close"][1]
        self.buying_date = df["date"][1]

        self.loss_cut_price = self.buying_price * (1 - self.max_loss_rate)
        self.profit_fixed_price = self.buying_price * (1 + self.sell_rate)
        self.index = 1
        self.df = df
        self.src_df = src_df
        return self.buying_price

    def run_simulation(self) -> float:
        """ """
        # print(df["date"][index], self.target_selling_price, self.loss_cut_price)
        df = self.df
        index = self.index
        # 最大保持日数を超えた場合は売る
        if df["date"][index] - self.buying_date > datetime.timedelta(days=self.total_max_days):
            self.selling_date = df["date"][index]
            if self.reach_target_price:
                self.selling_price = (
                    self.target_selling_price + min(self.loss_cut_price, df["open"][index])
                ) * 0.5
            else:
                self.selling_price = df["open"][index]
            return self.selling_price

        # 値上がりも値下がりもせず、一定期間過ぎた場合は売る
        if not self.reach_target_price and df["date"][
            index
        ] - self.buying_date > datetime.timedelta(days=self.max_days):
            self.selling_price = df["open"][index]
            self.selling_date = df["date"][index]
            return self.selling_price

        # 最大損失率を超えた場合は売る
        if df["low"][index] < self.loss_cut_price:
            self.selling_date = df["date"][index]
            if self.reach_target_price:
                self.selling_price = (
                    self.target_selling_price + min(self.loss_cut_price, df["open"][index])
                ) * 0.5
            else:
                self.selling_price = min(self.loss_cut_price, df["open"][index])
            return self.selling_price

        # ここまで値上がりしたら半分売る
        if df["high"][index] > self.profit_fixed_price and not self.reach_target_price:
            self.reach_target_price = True
            self.target_selling_price = max(self.profit_fixed_price, df["open"][index])

        # 十分値上がりしたらtrailling stop lossを適用
        if self.reach_target_price:
            self.loss_cut_price = max(
                self.loss_cut_price, df["high"][index] * (1 - self.trailling_stop_rate)
            )

        self.index += 1
        return -1.0


In [ ]:
# watch listの全銘柄に対してsimulationを実行
results = []
for index in tqdm(range(len(stacked_df))):
    #condition = StopCondition(sell_rate=0.01, max_loss_rate=0.08)
    condition = CustomStopCondition()
    res = stock.simulation.simulate.run(
        stacked_df["code"][index], stacked_df["date"][index], condition
    )
    results.append(res)

In [ ]:
profits = [res.profit for res in results]
sum(profits), sum(profits) / len(profits)

In [ ]:
profits = [res.profit for res in results]
sum(profits), sum(profits) / len(profits)

In [ ]:
plt.hist(profits, bins=100)

In [ ]:
index = 21
print(stacked_df["code"][index])
print(results[index].model_dump_json(indent=4))

In [ ]:
result_per_day = {}
for res in results:
    if res.buying_price < 0:
        continue
    if res.buying_date not in result_per_day:
        result_per_day[res.buying_date] = []
    result_per_day[res.buying_date].append(res.profit)

In [ ]:
# すべての銘柄に対して特定の月でのsimulationを実行
target_year = 2024
target_month = 7
date = datetime.date(target_year, target_month, 1)
code_list = stock.kabutan.get_code_list()

res_list = []
while date < datetime.date(target_year, target_month + 1, 1):
    for code in tqdm(code_list):
        condition = StopCondition(sell_rate=0.01, max_loss_rate=0.08)
        if stock.kabutan.data.calc_estimated_capitalization(code) > 100000000000:
            continue
        res = stock.simulation.simulate.run(code, date, condition)
        res_list.append(res)
    date += datetime.timedelta(days=1)

In [ ]:
res_list_per_day = [[] for _ in range(31)]
for res in res_list:
    if res.buying_price < 0 or res.buying_date.month > target_month:
        continue
    idx = res.buying_date.day - 1
    res_list_per_day[idx].append(res)

profits_per_day = {
    ress[0].buying_date: [res.profit for res in ress] for ress in res_list_per_day if len(ress) > 0
}
for date, profits in profits_per_day.items():
    if len(profits) > 0:
        print("date = {}, sum = {:>10.4f}, mean = {:>10.4f}".format(
            date, sum(profits), sum(profits) / len(profits)))
all_profits = [sum(profits) / len(profits) for _, profits in sorted(profits_per_day.items()) if len(profits) > 0]

In [ ]:
profits_per_day = {}
for res in results:
    if res.buying_price < 0:
        continue
    if res.buying_date.year == target_year and res.buying_date.month == target_month:
        target_date = res.buying_date
        if target_date not in profits_per_day:
            profits_per_day[target_date] = []
        profits_per_day[target_date].append(res.profit)

for date, profits in sorted(profits_per_day.items()):
    if len(profits) > 0:
        print("date = {}, sum = {:>10.4f}, mean = {:>10.4f}".format(
            date, sum(profits), sum(profits) / len(profits)))        
selected_profits = [sum(profits) / len(profits) for _, profits in sorted(profits_per_day.items()) if len(profits) > 0]

In [ ]:
sum(all_profits), sum(selected_profits)

In [ ]:
plt.plot(all_profits)
plt.plot(selected_profits)

In [ ]:
nasdaq = stock.scraping.read_data(stock.DATA_DIR / "us_data/nasdaq.csv")

In [ ]:
gap_downs = nasdaq.filter(
    ((pl.col("close") - pl.col("close").shift()) / pl.col("close").shift()) < -0.02
)

In [ ]:
nasdaq

In [ ]:
gap_downs

In [ ]:
result_on_gap_down = []
for idx in range(len(gap_downs)):
    gap_down_day = gap_downs["date"][idx] + datetime.timedelta(days=1)
    if gap_down_day not in result_per_day:
        continue
    result_on_gap_down.append(result_per_day[gap_down_day])

In [ ]:
profits_on_gap_down = [sum(res) / len(res) for res in result_on_gap_down]
sum(profits_on_gap_down) / len(profits_on_gap_down)

In [ ]:
# 大きく下がった銘柄は翌日に戻るのか
from stock.kabutan import read_data_csv, read_financial_csv

def calc_for_watch_list(
    code: str,
    start_date: datetime.date | None = None,
    end_date: datetime.date = datetime.date.today(),
):
    df = read_data_csv(code, start_date=start_date, end_date=end_date)
    # 過去10日の値動きの大きさを計算
    window_size = 10
    avg_key = "avg{}".format(window_size)
    stddev_key = "stddev{}".format(window_size)
    df = df.with_columns(
        pl.col("close").rolling_mean(window_size=window_size).alias(avg_key),
        pl.col("close").rolling_std(window_size=window_size).alias(stddev_key),
    )

    # ギャップダウンしている
    df = df.with_columns(
        (pl.col("close") < pl.col(avg_key) - pl.col(stddev_key)).alias("breakpoint")
    )

    # 直近の安値が安すぎない & 値幅が狭すぎない
    # window_size = 10
    # df = df.with_columns(
    #     (pl.col("close").rolling_min(window_size=window_size)).alias("min_close")
    # ).with_columns(
    #     (
    #         (pl.col("min_close") > pl.col("close") * 0.7)
    #         & (pl.col("min_close") < pl.col("close") * 0.95)
    #     ).alias("price_range")
    # )

    # 高値が多すぎない
    # df = df.with_columns(
    #     pl.col("close")
    #     .rolling_map(
    #         function=lambda d: sum(d > d[-1]),
    #         window_size=30,
    #     )
    #     .alias("high_count")
    # )

    # 出来高が増加（急増）
    window_size = 10
    df = df.with_columns(
        pl.col("volume").rolling_max(window_size=window_size).shift().alias("max_volume")
    )
    df = df.with_columns(
        (
            (pl.col("volume") > pl.col("max_volume") * 2)
            & (pl.col("volume") * pl.col("close") > 20000 * 100)
            & (pl.col("volume").rolling_max(window_size=30).shift() * 0.9 < pl.col("volume"))
        ).alias("volume_increase")
    )

    # watch listの条件判定
    df = df.with_columns(
        (
            pl.col("breakpoint")
            # & pl.col("price_range")
            & pl.col("volume_increase")
            # & (pl.col("high_count") < 7)
            & ((pl.col("close") >= pl.col("open")) | (pl.col("volume") > pl.col("max_volume") * 20))
        ).alias("watch_list")
    )

    # 直前にwatch list候補になっている場合はwatch listから除く
    df = df.with_columns(
        (
            (pl.col("watch_list").cast(int).rolling_max(window_size=5).shift() == 0)
            & pl.col("watch_list")
        ).alias("watch_list")
    )

    # 決算発表前後の日はwatch_listから除く
    fdf = (
        read_financial_csv(code)
        .filter(pl.col("annoounce_date") <= end_date)
        .sort(pl.col("annoounce_date"))
    )
    for announce_date in fdf["annoounce_date"]:
        df = df.with_columns(
            (
                pl.col("watch_list")
                & (
                    ~pl.col("date").is_between(
                        announce_date - datetime.timedelta(7), announce_date + datetime.timedelta(7)
                    )
                )
            ).alias("watch_list")
        )
    return df

In [ ]:
for code in code_list:
    df = calc_for_watch_list(code)
    break

In [ ]:
condition = StopCondition(sell_rate=0.01, max_loss_rate=0.08)
tmp = [stock.simulation.simulate.run("0000", df["date"][i], condition) for i in range(len(df))]

In [ ]:
df = stock.kabutan.read_data_csv("0000")

In [ ]:
df = df.with_columns(
    ((pl.col("open") - pl.col("close").shift()) / pl.col("open")).alias("gap"),
)

In [ ]:
df["open"][0], df["open"][-1]

In [ ]:
df = df.with_columns(
    ((pl.col("close") - pl.col("open")) / pl.col("open")).alias("gap"),
)

In [ ]:
tdf = df.select(
    pl.reduce(function = lambda acc, x: acc * x, exprs=pl.col("gap") + 1).alias("reduce")
)

In [ ]:
tdf

In [ ]:
(1 + df["gap"]) * df["open"][0]

In [ ]:
(1 + df["gap"].sum()) * df["open"][0]

In [ ]:
sum([d.profit for d in tmp])

In [ ]:
df = read_data_csv(code)

In [ ]:
ratios = []
for code in code_list:
    df = read_data_csv(code)
    ratios.append((df["close"] / df["open"]).sum() / len(df))